In [1]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from functools import reduce
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, BatchNormalization, Bidirectional, LayerNormalization, ConvLSTM1D, Flatten, Dropout
from keras.optimizers import Adam
import tensorflow as tf
import calendar

USE_GPU = 1
os.environ['CUDA_VISIBLE_DEVICES'] = '0' if USE_GPU else ''

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Constants
MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']



2024-06-15 09:13:44.214352: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 09:13:44.214389: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 09:13:44.214414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-15 09:13:44.222363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1


2024-06-15 09:13:45.620751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-15 09:13:45.626495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-15 09:13:45.626773: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
import pandas as pd
from functools import reduce

def split_by_param(df : pd.DataFrame, param : str) -> dict:
    """
    Splits the DataFrame based on the PARAM column into separate DataFrames for each unique PARAM value.
    """
    return {p: df[df[param] == p].reset_index(drop=True) for p in df[param].unique()}

def is_valid_date(year, month, day):
    """
     check if day > last day of specific month + year
    """
    try:
        m = calendar.monthrange(year, month)
        if day > m[1]:
            return False
        else:
            return True
    except Exception as e:
        print(e)
        return False
    
def convert_to_timeseries(df, month_identifiers : list = MONTHS):
    """
    Converts a DataFrame from wide to timeseries format.
    1999 Day1 JAN FEB MAR ...
    1999 Day2 JAN FEB MAR ...
    """
    data_dict = {'timestamp': [], 'value': []}

    for i, row in df.iterrows():
        year = row['YEAR']
        day = row['DD']
        for month in month_identifiers :
            valid_date = is_valid_date(year, month_identifiers.index(month) + 1, day)

            if valid_date:
                timestamp = pd.Timestamp(f'{year}-{month}-{day}')
                val = df.at[i, f'{month}']
                data_dict['timestamp'].append(timestamp)
                data_dict['value'].append(val)

    new_data = pd.DataFrame(data_dict)
    print(new_data['timestamp'].diff().dt.days.unique())
    return new_data

def merge_timeseries_dataframes(dataframes):

    return reduce(lambda left, right: pd.merge(left, right, on='timestamp', how='inner'), dataframes)



In [3]:

column_names = [
    'ID', 'PARAM', 'TYPE', 'YEAR', 'DD',
    'Jan', 'Jan_SYM', 'Feb', 'Feb_SYM', 'Mar', 'Mar_SYM',
    'Apr', 'Apr_SYM', 'May', 'May_SYM', 'Jun', 'Jun_SYM',
    'Jul', 'Jul_SYM', 'Aug', 'Aug_SYM', 'Sep', 'Sep_SYM',
    'Oct', 'Oct_SYM', 'Nov', 'Nov_SYM', 'Dec', 'Dec_SYM'
]
data = pd.read_csv('data/credit_hydrometric_data_all_stations.csv', skiprows=2, names=column_names)

station_data_split = {station: data[data['ID'] == station] for station in data['ID'].unique()}

In [4]:

# simple propagating fill in both directions
for station_id, station_data in station_data_split.items():
    station_data_split[station_id] = station_data.ffill().bfill()
    assert station_data_split[station_id].isnull().sum().sum() == 0

# Daily Discharge (m3/s) (PARAM = 1) and Daily Water Level (m) (PARAM = 2)
timeseries_station_params_split = {}
for station_id, station_data in station_data_split.items():
    params_split = split_by_param(station_data, 'PARAM')
    timeseries_station_params_split[station_id] = {param: convert_to_timeseries(params_split[param], MONTHS) for param in params_split}


[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   29.   30. -334.   60.   61.    1.   28. -333.   59.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.  366.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.  366.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]
[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.]


In [5]:


# rename columns to include station_id and param_name
for station_id, params_timeseries in timeseries_station_params_split.items():
    for param_name, params_timeseries in params_timeseries.items():
        if param_name == 1:
            name = 'discharge'
        elif param_name == 2:
            name = 'water_level'
        params_timeseries.rename(columns={'value': f'{name}_{station_id}'}, inplace=True)

In [6]:
all_dfs = [df for params_timeseries in timeseries_station_params_split.values() for df in params_timeseries.values()]

In [7]:

# Merge all timeseries dataframes
df_merged = merge_timeseries_dataframes(all_dfs)

print(df_merged['timestamp'].diff().dt.days.unique())

#df_merged.set_index('timestamp', inplace=True)

df_merged.sort_values(by='timestamp', inplace=True)



[  nan   31.   28.   30. -333.   59.   61.    1.   29. -334.   60.  366.]


In [8]:
def print_rows_with_gap(df):
    prev_timestamp = None
    prev_row = None

    for i, row in df.iterrows():
        current_timestamp = row['timestamp']

        if prev_timestamp is not None:
            diff = (current_timestamp - prev_timestamp).days
            if diff > 1:
                print(f'Previous row: {prev_row.timestamp}')
                print(f'Current row: {row.timestamp}')

        prev_timestamp = current_timestamp
        prev_row = row

# Usage:
# df is your DataFrame
print_rows_with_gap(df_merged)

Previous row: 2014-12-31 00:00:00
Current row: 2016-01-01 00:00:00


In [14]:
print(df_merged['timestamp'].diff().dt.days.unique())

[ nan   1. 366.]


In [15]:
# split df_merged on year >= 2014-12-31 00:00:00 and <2016-01-01 00:00:00
df_14 = df_merged[df_merged['timestamp'] < '2014-12-31']
df_16= df_merged[df_merged['timestamp'] >= '2016-01-01']

In [16]:
print(df_14['timestamp'].diff().dt.days.unique())
print(df_16['timestamp'].diff().dt.days.unique())

[nan  1.]
[nan  1.]


In [11]:
def load_data(filepath):
    column_names = [
        'ID', 'PARAM', 'TYPE', 'YEAR', 'DD',
        'Jan', 'Jan_SYM', 'Feb', 'Feb_SYM', 'Mar', 'Mar_SYM',
        'Apr', 'Apr_SYM', 'May', 'May_SYM', 'Jun', 'Jun_SYM',
        'Jul', 'Jul_SYM', 'Aug', 'Aug_SYM', 'Sep', 'Sep_SYM',
        'Oct', 'Oct_SYM', 'Nov', 'Nov_SYM', 'Dec', 'Dec_SYM'
    ]
    data = pd.read_csv(filepath, skiprows=2, names=column_names)
    return data

def preprocess_data(data, station_id):
    df = data[data['ID'] == station_id].reset_index(drop=True)
    return df

def convert_to_timeseries(df, value_column='value'):
    # Conversion logic here
    pass

def merge_datasets(list_of_datasets):
    return reduce(lambda left, right: pd.merge(left, right, on=['timestamp'], how='outer'), list_of_datasets)

def add_lag_times(df, lag_times):
    df_copy = df.copy()
    for column, lag in lag_times.items():
        df_copy[f'{column}_lagged'] = df[column].shift(lag)
    return df_copy

def interpolate_hourly(df):
    return df.resample('6H').interpolate('linear')


In [12]:
def make_lstm(input_shape):
    model = Sequential([
        Bidirectional(LSTM(100, return_sequences=True, input_shape=input_shape, dropout=0.2, recurrent_dropout=0.2)),
        BatchNormalization(),
        Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2)),
        BatchNormalization(),
        Dense(1)
    ])
    return model

def make_gru(input_shape):
    model = Sequential([
        GRU(50, return_sequences=True, input_shape=input_shape, dropout=0.2),
        LayerNormalization(),
        GRU(25, dropout=0.2),
        Dense(1)
    ])
    return model

def make_conv_lstm(input_shape):
    model = Sequential([
        ConvLSTM1D(filters=32, kernel_size=3, return_sequences=True, input_shape=input_shape),
        BatchNormalization(),
        Flatten(),
        Dense(50, activation='relu'),
        Dropout(0.25),
        Dense(1)
    ])
    return model

In [13]:
datasets = {
    'merged': df_merged,
    'hourly': df_hourly,
    'lagged': df_hourly_lagged
}

models = {
    'LSTM': make_lstm,
    'GRU': make_gru,
    'ConvLSTM': make_conv_lstm
}

results = {}

for dataset_name, df in datasets.items():
    X_train, X_test, Y_train, Y_test = prepare_dataset(df)  # Define this function based on your dataset structure
    for model_name, model_builder in models.items():
        model = model_builder((X_train.shape[1],))
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
        history = model.fit(X_train, Y_train, validation_split=0.2, epochs=10, batch_size=256, verbose=2)
        eval

NameError: name 'df_hourly' is not defined